In [ ]:
import os
import sys
import glob
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

fontsize = 8
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

%matplotlib inline

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments

In [ ]:
def make_axes(rows, cols, x_offset, y_offset, x_space, y_space, squeeze=True):
    w = (1 - np.sum(x_offset) - x_space * (cols - 1)) / cols
    h = (1 - np.sum(y_offset) - y_space * (rows - 1)) / rows
    
    ax = [[plt.axes([x_offset[0] + (w + x_space) * j,
                     y_offset[0] + (h + y_space) * i,
                     w, h]) for j in range(cols)] for i in range(rows-1, -1, -1)]
    
    for row in ax:
        for a in row:
            for side in 'right','top':
                a.spines[side].set_visible(False)

    if squeeze:
        if rows == 1 and cols == 1:
            return ax[0][0]
        if rows == 1:
            return ax[0]
        if cols == 1:
            return [a[0] for a in ax]
        
    return ax

### Figure for the PSCC-2022 abstract

In [ ]:
area_ID = 1
experiments = collect_experiments(area_ID, area_measure='energy', D=2, DZA=60)

experiment_IDs = list(experiments.keys())
experiment_ID = experiment_IDs[np.argmin([expt['loss'].min() for expt in experiments.values()])]
MAPE = experiments[experiment_ID]['MAPE']
loss = experiments[experiment_ID]['loss']
val_loss = experiments[experiment_ID]['val_loss']
batch_loss = experiments[experiment_ID]['batch_loss']
tags = experiments[experiment_ID]['tags']
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE}%).')

experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(experiments_path + experiment_ID + '/parameters.pkl', 'rb'))
test_results = pickle.load(open(experiments_path + experiment_ID + '/test_results.pkl', 'rb'))

In [ ]:
n_epochs = len(loss)
epochs = np.arange(n_epochs) + 1

font = {'family' : 'Times New Roman',
        'size'   : 8}
matplotlib.rc('font', **font)

fig,ax = plt.subplots(1, 2, figsize=(3.3, 1.8))
ax[0].semilogy(epochs, val_loss, color=[.8,.8,.8], lw=1, label='Validation set')
ax[0].semilogy(epochs, loss, 'k', lw=1, label='Training set')
ax[0].legend(loc='upper right', fontsize=7)
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('MAE')
ax[0].set_xticks(np.r_[0 : 650 : 150])
y_test, y_prediction = test_results['y_test'], test_results['y_prediction']
ax[1].plot([8,14.5], [8,14.5], lw=2, color=[.8,.8,.8])
ax[1].plot(y_test, y_prediction, 'o', color=[.6,.6,.6], markerfacecolor='w', markersize=3, markeredgewidth=0.5)
for x in np.unique(y_test):
    idx, = np.where(y_test == x)
    ymean, ystd = y_prediction[idx].mean(), y_prediction[idx].std()
    ysem = ystd / np.sqrt(len(idx))
    ax[1].plot(x + np.zeros(2), ymean + 3 * ystd * np.array([-1,1]), 'k', linewidth=1)
    ax[1].plot(x, ymean, 'ko', markerfacecolor='w', markersize=3.75, markeredgewidth=1)
ax[1].set_xlabel(r'Inertia [GW$\cdot$s]')
ax[1].set_ylabel(r'Predicted inertia [GW$\cdot$s]')
ax[1].set_xticks(np.r_[8:16])
ax[1].set_yticks(np.r_[8:16])
ax[1].text(8.5, 14.5, f'MAPE = {MAPE:.2f}%', fontsize=7)

for a in ax:
    for side in 'top','right':
        a.spines[side].set_visible(False)

fig.tight_layout(pad=0.1)
fig.savefig(f'training_results_{experiment_ID[:6]}.pdf')